In [8]:
import pandas as pd
import numpy as np
import time
import os
import threading
from collections import deque
import warnings
warnings.filterwarnings('ignore')

# Deep Learning and Model Loading
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer

# MIDI and Audio Libraries
import pretty_midi
import mido
import pygame
from scipy.io import wavfile
import soundfile as sf

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class CustomAttention(Layer):
    """Custom attention mechanism as a proper Keras layer"""
    
    def __init__(self, attention_dim=64, **kwargs):
        super(CustomAttention, self).__init__(**kwargs)
        self.attention_dim = attention_dim
        
    def build(self, input_shape):
        self.input_dim = input_shape[-1]
        self.query_dense = tf.keras.layers.Dense(self.attention_dim, activation='tanh')
        self.key_dense = tf.keras.layers.Dense(self.attention_dim, activation='tanh')
        self.value_dense = tf.keras.layers.Dense(self.input_dim, activation='tanh')
        super(CustomAttention, self).build(input_shape)
        
    def call(self, inputs):
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        
        attention_scores = tf.keras.backend.batch_dot(query, key, axes=[2, 2])
        attention_scores = tf.nn.softmax(attention_scores, axis=-1)
        
        attended = tf.keras.backend.batch_dot(attention_scores, value, axes=[2, 1])
        attended = attended + inputs
        
        return attended
    
    def get_config(self):
        config = super(CustomAttention, self).get_config()
        config.update({"attention_dim": self.attention_dim})
        return config

class SensorDataBuffer:
    """Manages sliding window of sensor data"""
    
    def __init__(self, sequence_length=20, feature_dim=9):
        self.sequence_length = sequence_length
        self.feature_dim = feature_dim
        self.buffer = deque(maxlen=sequence_length)
        self.feature_means = None
        self.feature_stds = None
        
    def add_data_point(self, data_point):
        """Add new data point to buffer"""
        if len(data_point) != self.feature_dim:
            raise ValueError(f"Data point must have {self.feature_dim} features")
        self.buffer.append(data_point)
        
    def get_sequence(self):
        """Get current sequence for model inference"""
        if len(self.buffer) < self.sequence_length:
            # Pad with zeros if not enough data
            padded_data = np.zeros((self.sequence_length, self.feature_dim))
            available_data = np.array(list(self.buffer))
            padded_data[-len(available_data):] = available_data
            return padded_data
        else:
            return np.array(list(self.buffer))
    
    def normalize_sequence(self, sequence):
        """Normalize sequence similar to training data"""
        if self.feature_means is None or self.feature_stds is None:
            # Initialize normalization parameters
            self.feature_means = np.mean(sequence, axis=0)
            self.feature_stds = np.std(sequence, axis=0) + 1e-8
        
        normalized = (sequence - self.feature_means) / self.feature_stds
        return normalized
    
    def is_ready(self):
        """Check if buffer has enough data for inference"""
        return len(self.buffer) >= self.sequence_length

class WeatherToMusicMapper:
    """Maps weather predictions to musical parameters"""
    
    def __init__(self):
        self.weather_classes = ['Sunny', 'Rainy', 'Stormy', 'Windy']
        
        # Musical mapping dictionary
        self.music_mapping = {
            'Sunny': {
                'key': 'C',
                'scale': 'major',
                'tempo': 120,
                'instrument': 1,  # Piano
                'chord_type': 'major',
                'base_note': 60,  # C4
                'velocity': 80,
                'rhythm_pattern': [1, 0, 0.5, 0, 1, 0, 0.5, 0],
                'chord_progression': [60, 64, 67, 72],  # C, E, G, C
                'color': 'bright'
            },
            'Rainy': {
                'key': 'Dm',
                'scale': 'minor',
                'tempo': 90,
                'instrument': 48,  # String ensemble
                'chord_type': 'minor',
                'base_note': 62,  # D4
                'velocity': 60,
                'rhythm_pattern': [1, 0, 0, 0, 0.5, 0, 0, 0],
                'chord_progression': [62, 65, 69, 74],  # D, F, A, D
                'color': 'melancholic'
            },
            'Stormy': {
                'key': 'G',
                'scale': 'diminished',
                'tempo': 140,
                'instrument': 32,  # Distortion Guitar
                'chord_type': 'diminished',
                'base_note': 55,  # G3
                'velocity': 100,
                'rhythm_pattern': [1, 0.5, 1, 0.5, 1, 0.5, 1, 0.5],
                'chord_progression': [55, 58, 61, 64],  # G, Bb, Db, E
                'color': 'dramatic'
            },
            'Windy': {
                'key': 'Am',
                'scale': 'aeolian',
                'tempo': 110,
                'instrument': 73,  # Flute
                'chord_type': 'minor',
                'base_note': 57,  # A3
                'velocity': 70,
                'rhythm_pattern': [0.5, 0.5, 1, 0, 0.5, 0.5, 1, 0],
                'chord_progression': [57, 60, 64, 67],  # A, C, E, G
                'color': 'airy'
            }
        }
    
    def get_musical_parameters(self, prediction_class, confidence=1.0):
        """Get musical parameters for a weather prediction"""
        if prediction_class >= len(self.weather_classes):
            prediction_class = 0
        
        weather = self.weather_classes[prediction_class]
        params = self.music_mapping[weather].copy()
        
        # Modify parameters based on confidence
        params['velocity'] = int(params['velocity'] * confidence)
        params['tempo'] = int(params['tempo'] * (0.8 + 0.4 * confidence))
        
        return params, weather

class MIDIGenerator:
    """Generates and manages MIDI output"""
    
    def __init__(self, output_dir="MIDI"):
        self.output_dir = output_dir
        self.ensure_output_dir()
        
        # Initialize MIDI
        self.midi_file = pretty_midi.PrettyMIDI()
        self.current_time = 0
        self.note_duration = 0.5
        
        # Track different instruments
        self.instruments = {}
        
        # For smoothing transitions
        self.last_prediction = None
        self.prediction_history = deque(maxlen=5)
        
    def ensure_output_dir(self):
        """Create output directory if it doesn't exist"""
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
            print(f"Created output directory: {self.output_dir}")
    
    def get_or_create_instrument(self, program, name):
        """Get or create instrument for MIDI"""
        if name not in self.instruments:
            instrument = pretty_midi.Instrument(program=program, name=name)
            self.instruments[name] = instrument
            self.midi_file.instruments.append(instrument)
        return self.instruments[name]
    
    def smooth_prediction(self, new_prediction):
        """Smooth predictions to avoid abrupt changes"""
        self.prediction_history.append(new_prediction)
        
        if len(self.prediction_history) < 3:
            return new_prediction
        
        # Use majority voting for smoothing
        recent_predictions = list(self.prediction_history)[-3:]
        prediction_counts = {}
        
        for pred in recent_predictions:
            prediction_counts[pred] = prediction_counts.get(pred, 0) + 1
        
        # Return most common prediction
        smoothed_prediction = max(prediction_counts, key=prediction_counts.get)
        return smoothed_prediction
    
    def generate_chord(self, base_note, chord_type, velocity=80):
        """Generate chord based on base note and type"""
        if chord_type == 'major':
            intervals = [0, 4, 7]
        elif chord_type == 'minor':
            intervals = [0, 3, 7]
        elif chord_type == 'diminished':
            intervals = [0, 3, 6]
        else:
            intervals = [0, 4, 7]  # Default to major
        
        chord_notes = [base_note + interval for interval in intervals]
        return chord_notes
    
    def add_musical_phrase(self, params, weather_name, transition_smooth=True):
        """Add musical phrase to MIDI"""
        # Get or create instrument
        instrument = self.get_or_create_instrument(
            params['instrument'], 
            f"{weather_name}_instrument"
        )
        
        # Generate chord
        chord_notes = self.generate_chord(
            params['base_note'], 
            params['chord_type'], 
            params['velocity']
        )
        
        # Add chord notes
        for note_pitch in chord_notes:
            note = pretty_midi.Note(
                velocity=params['velocity'],
                pitch=note_pitch,
                start=self.current_time,
                end=self.current_time + self.note_duration
            )
            instrument.notes.append(note)
        
        # Add melody note
        melody_note = pretty_midi.Note(
            velocity=params['velocity'] + 20,
            pitch=params['base_note'] + 12,  # Octave higher
            start=self.current_time,
            end=self.current_time + self.note_duration
        )
        instrument.notes.append(melody_note)
        
        # Update time
        beat_duration = 60.0 / params['tempo']
        self.current_time += beat_duration
    
    def save_midi_file(self, filename="sensor_music.mid"):
        """Save MIDI file"""
        filepath = os.path.join(self.output_dir, filename)
        self.midi_file.write(filepath)
        print(f"MIDI file saved: {filepath}")
        return filepath
    
    def convert_to_audio(self, midi_filepath, audio_filename="sensor_music.wav"):
        """Convert MIDI to audio using synthesizer"""
        try:
            # Load MIDI file
            midi_data = pretty_midi.PrettyMIDI(midi_filepath)
            
            # Synthesize audio
            audio = midi_data.synthesize(fs=44100)
            
            # Save as WAV
            audio_filepath = os.path.join(self.output_dir, audio_filename)
            sf.write(audio_filepath, audio, 44100)
            print(f"Audio file saved: {audio_filepath}")
            
            return audio_filepath
            
        except Exception as e:
            print(f"Error converting MIDI to audio: {e}")
            return None

class RealTimeMusicSystem:
    """Main system orchestrating sensor data to music conversion"""
    
    def __init__(self, model_path="best_model.h5", sequence_length=20, feature_dim=9):
        self.sequence_length = sequence_length
        self.feature_dim = feature_dim
        
        # Initialize components
        self.data_buffer = SensorDataBuffer(sequence_length, feature_dim)
        self.music_mapper = WeatherToMusicMapper()
        self.midi_generator = MIDIGenerator()
        
        # Load model
        self.model = self.load_model(model_path)
        
        # System state
        self.is_running = False
        self.prediction_count = 0
        self.current_weather = None
        
        # Performance metrics
        self.inference_times = []
        self.predictions_history = []
        
    def load_model(self, model_path):
        """Load the pre-trained model"""
        try:
            # Register custom objects
            custom_objects = {'CustomAttention': CustomAttention}
            model = load_model(model_path, custom_objects=custom_objects)
            print(f"Model loaded successfully from {model_path}")
            print(f"Model input shape: {model.input_shape}")
            print(f"Model output shape: {model.output_shape}")
            return model
        except Exception as e:
            print(f"Error loading model: {e}")
            return None
    
    def load_sensor_data(self, filepath="midiMusic.csv"):
        """Load sensor data from CSV file"""
        try:
            df = pd.read_csv(filepath)
            print(f"Loaded sensor data: {df.shape}")
            print(f"Columns: {df.columns.tolist()}")
            
            # Extract feature columns
            feature_cols = ['Timestamp','Temperature', 'Humidity', 'Acc_X', 'Acc_Y', 'Acc_Z', 
                           'Gyro_X', 'Gyro_Y', 'Gyro_Z', 'LDR']
            
            if all(col in df.columns for col in feature_cols):
                sensor_data = df[feature_cols].values
                print(f"Extracted {len(sensor_data)} sensor readings")
                return sensor_data
            else:
                print("Required feature columns not found in CSV")
                return None
                
        except Exception as e:
            print(f"Error loading sensor data: {e}")
            return None
    
    def predict_weather(self, sensor_sequence):
        """Make weather prediction from sensor sequence"""
        if self.model is None:
            return None, None
        
        try:
            start_time = time.time()
            
            # Normalize sequence
            normalized_sequence = self.data_buffer.normalize_sequence(sensor_sequence)
            
            # Reshape for model input
            input_data = normalized_sequence.reshape(1, self.sequence_length, self.feature_dim)
            
            # Make prediction
            prediction_probs = self.model.predict(input_data, verbose=0)
            predicted_class = np.argmax(prediction_probs[0])
            confidence = np.max(prediction_probs[0])
            
            # Record inference time
            inference_time = time.time() - start_time
            self.inference_times.append(inference_time)
            
            return predicted_class, confidence
            
        except Exception as e:
            print(f"Error in weather prediction: {e}")
            return None, None
    
    def process_sensor_data_point(self, data_point):
        """Process single sensor data point"""
        # Add to buffer
        self.data_buffer.add_data_point(data_point)
        
        # Check if ready for inference
        if not self.data_buffer.is_ready():
            return None
        
        # Get sequence and make prediction
        sequence = self.data_buffer.get_sequence()
        predicted_class, confidence = self.predict_weather(sequence)
        
        if predicted_class is not None:
            # Smooth prediction
            smoothed_prediction = self.midi_generator.smooth_prediction(predicted_class)
            
            # Get musical parameters
            params, weather_name = self.music_mapper.get_musical_parameters(
                smoothed_prediction, confidence
            )
            
            # Generate music
            self.midi_generator.add_musical_phrase(params, weather_name)
            
            # Update state
            self.current_weather = weather_name
            self.prediction_count += 1
            self.predictions_history.append((weather_name, confidence))
            
            return {
                'weather': weather_name,
                'confidence': confidence,
                'prediction_count': self.prediction_count,
                'musical_params': params
            }
        
        return None
    
    def simulate_real_time_stream(self, sensor_data, delay=0.1):
        """Simulate real-time sensor data stream"""
        print(f"Starting real-time simulation with {len(sensor_data)} data points")
        print(f"Delay between readings: {delay} seconds")
        
        self.is_running = True
        
        for i, data_point in enumerate(sensor_data):
            if not self.is_running:
                break
            
            # Process data point
            result = self.process_sensor_data_point(data_point)
            
            if result:
                print(f"Step {i+1}: {result['weather']} "
                      f"(confidence: {result['confidence']:.3f}, "
                      f"tempo: {result['musical_params']['tempo']})")
            
            # Wait before next reading
            time.sleep(delay)
        
        print("Real-time simulation completed")
    
    def generate_music_visualization(self):
        """Generate visualization of music generation process"""
        if not self.predictions_history:
            print("No predictions to visualize")
            return
        
        # Create visualization
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Weather predictions over time
        weather_names = [pred[0] for pred in self.predictions_history]
        confidences = [pred[1] for pred in self.predictions_history]
        
        # Plot weather timeline
        axes[0, 0].plot(weather_names, 'o-', markersize=6)
        axes[0, 0].set_title('Weather Predictions Timeline')
        axes[0, 0].set_xlabel('Time Steps')
        axes[0, 0].set_ylabel('Weather Class')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Plot confidence levels
        axes[0, 1].plot(confidences, 'g-', linewidth=2)
        axes[0, 1].set_title('Prediction Confidence Over Time')
        axes[0, 1].set_xlabel('Time Steps')
        axes[0, 1].set_ylabel('Confidence')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Weather distribution
        weather_counts = {}
        for weather in weather_names:
            weather_counts[weather] = weather_counts.get(weather, 0) + 1
        
        axes[1, 0].bar(weather_counts.keys(), weather_counts.values())
        axes[1, 0].set_title('Weather Class Distribution')
        axes[1, 0].set_xlabel('Weather Class')
        axes[1, 0].set_ylabel('Count')
        
        # Inference time performance
        if self.inference_times:
            axes[1, 1].plot(self.inference_times, 'r-', alpha=0.7)
            axes[1, 1].set_title('Model Inference Time')
            axes[1, 1].set_xlabel('Prediction Number')
            axes[1, 1].set_ylabel('Time (seconds)')
            axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(self.midi_generator.output_dir, 'music_generation_analysis.png'))
        plt.show()
    
    def generate_final_music_file(self):
        """Generate final music files"""
        print("\nGenerating final music files...")
        
        # Save MIDI file
        midi_filepath = self.midi_generator.save_midi_file("sensor_generated_music.mid")
        
        # Convert to audio
        audio_filepath = self.midi_generator.convert_to_audio(
            midi_filepath, 
            "sensor_generated_music.wav"
        )
        
        # Generate summary report
        self.generate_summary_report()
        
        return midi_filepath, audio_filepath
    
    def generate_summary_report(self):
        """Generate summary report of the music generation process"""
        report_filepath = os.path.join(self.midi_generator.output_dir, "generation_report.txt")
        
        with open(report_filepath, 'w') as f:
            f.write("SENSOR-TO-MUSIC GENERATION REPORT\n")
            f.write("="*50 + "\n\n")
            
            f.write(f"Total predictions made: {self.prediction_count}\n")
            f.write(f"Total music duration: {self.midi_generator.current_time:.2f} seconds\n\n")
            
            if self.predictions_history:
                f.write("WEATHER PREDICTIONS SUMMARY:\n")
                weather_counts = {}
                for weather, _ in self.predictions_history:
                    weather_counts[weather] = weather_counts.get(weather, 0) + 1
                
                for weather, count in weather_counts.items():
                    percentage = (count / len(self.predictions_history)) * 100
                    f.write(f"  {weather}: {count} ({percentage:.1f}%)\n")
            
            f.write(f"\nAverage inference time: {np.mean(self.inference_times):.4f} seconds\n")
            f.write(f"Musical instruments used: {len(self.midi_generator.instruments)}\n")
            
            f.write("\nINSTRUMENT MAPPING:\n")
            for weather, params in self.music_mapper.music_mapping.items():
                f.write(f"  {weather}: Instrument {params['instrument']}, "
                       f"Key {params['key']}, Tempo {params['tempo']}\n")
        
        print(f"Summary report saved: {report_filepath}")
    
    def run_complete_pipeline(self, sensor_data_file="midiMusic.csv", simulation_delay=0.05):
        """Run the complete sensor-to-music pipeline"""
        print("="*60)
        print("REAL-TIME SENSOR-TO-MUSIC CONVERSION SYSTEM")
        print("="*60)
        
        # Load sensor data
        sensor_data = self.load_sensor_data(sensor_data_file)
        if sensor_data is None:
            print("Failed to load sensor data. Exiting.")
            return
        
        # Simulate real-time processing
        self.simulate_real_time_stream(sensor_data, delay=simulation_delay)
        
        # Generate visualizations
        self.generate_music_visualization()
        
        # Generate final music files
        midi_file, audio_file = self.generate_final_music_file()
        
        print("\n" + "="*60)
        print("PIPELINE COMPLETED SUCCESSFULLY!")
        print("="*60)
        print(f"MIDI file: {midi_file}")
        print(f"Audio file: {audio_file}")
        print(f"Total predictions: {self.prediction_count}")
        print(f"Music duration: {self.midi_generator.current_time:.2f} seconds")
        
        return midi_file, audio_file

def main():
    """Main execution function"""
    # Initialize the system
    system = RealTimeMusicSystem(
        model_path="best_model.h5",
        sequence_length=20,
        feature_dim=9
    )
    
    # Run the complete pipeline
    midi_file, audio_file = system.run_complete_pipeline(
        sensor_data_file="midiMusic.csv",
        simulation_delay=0.02  # Fast simulation for demonstration
    )
    
    print("\nSystem ready for real-time operation!")
    print("Files generated in 'MIDI' directory:")
    print("- sensor_generated_music.mid (MIDI file)")
    print("- sensor_generated_music.wav (Audio file)")
    print("- music_generation_analysis.png (Visualization)")
    print("- generation_report.txt (Summary report)")

if __name__ == "__main__":
    main()

Model loaded successfully from best_model.h5
Model input shape: (None, 20, 9)
Model output shape: (None, 4)
REAL-TIME SENSOR-TO-MUSIC CONVERSION SYSTEM
Loaded sensor data: (2273, 10)
Columns: ['16:43:43', '0.849264705882353', '0.1866666666666666', '0.034313725490196', '0.5076142131979695', '0.1735159817351598', '0.4686246810061975', '0.480494966442953', '0.3984025793214626', '0.0909090909090909']
Required feature columns not found in CSV
Failed to load sensor data. Exiting.


TypeError: cannot unpack non-iterable NoneType object